# MarketHours class

One of the biggest pains when dealing with trading data is handling the timestamps. This imposes many challenges due to:
- Missing dates.
- Holidays 
- Daytime Light Savings change.
- Intraday plottings are inneficient if you plot.
- Missalignment between trading natural days and the start of trading sessions.


The MarketHours class help us handle all of this operations.

Every symbol has a regular market hours like for example:
- 8:00 - 17:00 for Europen stocks Monday-Friday
- 8:00 - 19:00 for Forex Monday-Saturday

This is the normal market hours but there can be several factors to change:
- DST: It changes abruptly the hours
- Holidays: Not open at all
- Special days: Just open a few hours.

Most of the functionalities of this class operate with intraday data. For daily candlesticks it can be used to know if a day should be trading or not.

In [1]:
import datetime as dt
import pandas as pd
from IPython.display import Image
%matplotlib qt

import sys
sys.path.append("..") # Adds higher directory to python modules path.

from traphing.data_classes import Velas
from traphing.utils import Timeframes, unwrap, MarketHours
from traphing.graph.Gl import gl

## Instance of the class


When instanciating the clase we can optionally include 

In [2]:
my_market_hours = MarketHours()

In [3]:
unwrap(my_market_hours)

<MarketHours>	object has children:
   <NoneType>	open_time:	None
   <NoneType>	close_time:	None
   <list>	trading_days_list
   <NoneType>	special_days_dict:	None

   <list>	trading_days_list has children:
      <int>	trading_days_list[0]:	0
      <int>	trading_days_list[1]:	1
      <int>	trading_days_list[2]:	2
      <int>	trading_days_list[3]:	3
      <int>	trading_days_list[4]:	4




## Interaction with Velas object

The MarketHours class is intended to interact with the data from Velas objects. The Velas object has a set of functionalities that make use of the methods in MarketHours transparently. In the following, we will show some examples of this interaction.

As always, we first need to import velas objects:

In [4]:
symbol_name = "AUDCHF"
timeframe = Timeframes.M15
storage_folder = "../tests/data/storage/"
start_time = dt.datetime(2019,7,20)
end_time = dt.datetime(2019,8,20)

my_velas_M15 = Velas(symbol_name, timeframe)
my_velas_M15.load_data_from_csv(storage_folder)
my_velas_M15.set_time_interval(start_time, end_time, trim = False)

timestamps_M15 = my_velas_M15.timestamps
dates_M15 = my_velas_M15.dates
unique_dates = dates_M15.unique()

Size ../tests/data/storage/M15/AUDCHF_M15.csv:  100400  rows


## The Iso calendar. The nightmare of time zones

- DST is more political than rational.
- The timestamps could be naive or aware regarding the timezone. If they are naive (no infomation), then UTC is assumed, so when calling isocalendar() we get????
- There is the  Coordinated Universal Time (UTC)
It is important to have a reference due to all the TimeZones, in our case we normalize to the ISO?

In [5]:
print ("Weekday and week number of ",timestamps_M15[0], timestamps_M15[0].tzname())
print ("Weekday = ", timestamps_M15[0].weekday())
print ("Week number = ", timestamps_M15[0].week)

Weekday and week number of  2019-07-22 00:00:00 None
Weekday =  0
Week number =  30


In [6]:
print(timestamps_M15[0].tz)
print(timestamps_M15[0])
import pytz 
from tzlocal import get_localzone # $ pip install tzlocal

# get local timezone    
local_tz = get_localzone()
local_tz = "UTC"
print(local_tz)
tz_aware_time = timestamps_M15[0].tz_localize(local_tz) #Convert naive Timestamp to local time zone, or remove timezone from tz-aware Timestamp.
print(tz_aware_time)
print(tz_aware_time.tz)
print(tz_aware_time.isocalendar())

None
2019-07-22 00:00:00
UTC
2019-07-22 00:00:00+00:00
UTC
(2019, 30, 1)


In [7]:
print ("ISO weekday and week number of ",timestamps_M15[0])
print ("Weekday = ", timestamps_M15[0].isocalendar()[2])
print ("Week number = ", timestamps_M15[0].isocalendar()[1])

ISO weekday and week number of  2019-07-22 00:00:00
Weekday =  1
Week number =  30


## Grouping by weekday number

In [8]:
my_market_hours.get_number_of_samples_by_weekday_dict(timestamps_M15)

{0: 480, 1: 480, 2: 384, 3: 384, 4: 384}

In [9]:
my_market_hours.get_number_of_samples_by_weekday_dict(unique_dates)

{0: 5, 1: 5, 2: 4, 3: 4, 4: 4}

## Grouping by natural days

In [10]:
daily_groups = MarketHours.get_index_by_days_dict(timestamps_M15)

print(daily_groups.keys())
for i in range(4):
    print (daily_groups[unique_dates[i]])

dict_keys([datetime.date(2019, 7, 22), datetime.date(2019, 7, 23), datetime.date(2019, 7, 24), datetime.date(2019, 7, 25), datetime.date(2019, 7, 26), datetime.date(2019, 7, 29), datetime.date(2019, 7, 30), datetime.date(2019, 7, 31), datetime.date(2019, 8, 1), datetime.date(2019, 8, 2), datetime.date(2019, 8, 5), datetime.date(2019, 8, 6), datetime.date(2019, 8, 7), datetime.date(2019, 8, 8), datetime.date(2019, 8, 9), datetime.date(2019, 8, 12), datetime.date(2019, 8, 13), datetime.date(2019, 8, 14), datetime.date(2019, 8, 15), datetime.date(2019, 8, 16), datetime.date(2019, 8, 19), datetime.date(2019, 8, 20)])
DatetimeIndex(['2019-07-22 00:00:00', '2019-07-22 00:15:00',
               '2019-07-22 00:30:00', '2019-07-22 00:45:00',
               '2019-07-22 01:00:00', '2019-07-22 01:15:00',
               '2019-07-22 01:30:00', '2019-07-22 01:45:00',
               '2019-07-22 02:00:00', '2019-07-22 02:15:00',
               '2019-07-22 02:30:00', '2019-07-22 02:45:00',
             

## Estimation time

Sometimes we will not be sure of the trading hours, or some shit. The things we can estimate are:
- timeframe: 
- open_time and close_time:
- usual_trading_days:
- irregular_days:

### timeframe

In [11]:
my_market_hours.estimate_timeframe(timestamps_M15)

<Timeframes.M15: 15>

### open_time and close_time

In [12]:
open_time, close_time = my_market_hours.estimate_open_close_time(timestamps_M15)
print ("Usual natural day market hours: ", str(open_time), " - ", str(close_time))

Usual natural day market hours:  00:00:00  -  00:00:00


### usual_trading_days

In [13]:
n_samples = my_market_hours.get_number_of_samples_per_trading_session(open_time,close_time, timeframe)
print ("Number of usual sample: ", n_samples)

Number of usual sample:  96


In [14]:
normal_trading_days_list = my_market_hours.estimate_normal_trading_days(timestamps_M15, open_time,close_time, timeframe)
print(normal_trading_days_list)

[0, 1, 2, 3, 4]


In [15]:
special_trading_days_dict = my_market_hours.estimate_special_trading_days_from_timestamps(timestamps_M15, open_time,close_time, timeframe, normal_trading_days_list)
for key in special_trading_days_dict.keys():
    print (special_trading_days_dict[key].close_time)

### Unwrapping the object again

After the estimations, the internal data of the object is also modified

In [20]:
unwrap(my_market_hours)

<MarketHours>	object has children:
   <time>	open_time
   <time>	close_time
   <list>	trading_days_list
   <dict>	special_days_dict
   <Timeframes>	timeframe:	Timeframes.M15

   <dict>	special_days_dict has children:
      <SpecialMarketHours>	2019-08-20

      <SpecialMarketHours>	2019-08-20 has children:
         <time>	open_time
         <time>	close_time
         <int>	n_samples:	86

         <time>	close_time has children:

         <time>	open_time has children:

   <list>	trading_days_list has children:
      <int>	trading_days_list[0]:	0
      <int>	trading_days_list[1]:	1
      <int>	trading_days_list[2]:	2
      <int>	trading_days_list[3]:	3
      <int>	trading_days_list[4]:	4

   <time>	close_time has children:

   <time>	open_time has children:




## Delete some samples to check functionality

In [16]:
timestamps_M15 = timestamps_M15[:-10]

In [17]:
special_trading_days_dict = my_market_hours.estimate_special_trading_days_from_timestamps(timestamps_M15, open_time,close_time, timeframe, normal_trading_days_list)
for key in special_trading_days_dict.keys():
    print("Date " + str(key) + " is different:")
    print("\t Usual open time: \t", open_time, "\t Special: ", str(special_trading_days_dict[key].open_time))
    print("\t Usual close time: \t", close_time, "\t Special: ", str(special_trading_days_dict[key].close_time))
    print("\t Usual n samples: \t", n_samples, "\t\t Special: ", str(special_trading_days_dict[key].n_samples))

Date 2019-08-20 is different:
	 Usual open time: 	 00:00:00 	 Special:  00:00:00
	 Usual close time: 	 00:00:00 	 Special:  21:30:00
	 Usual n samples: 	 96 		 Special:  86


## Use of the object
Once the properties of the days have been set or estimated we can use the object to easily obtain the information of everyday.

### Check if a day is special

In [18]:
for date in unique_dates:
    if (my_market_hours.is_special(date)):
        print ("Special date: ", date)

Special date:  2019-08-20


### Check if a day is a trading day

In [19]:
first_date = unique_dates[0]
last_date = unique_dates[-1]

date = first_date
while (date < last_date):
    date += dt.timedelta(days = 1)
    is_trading_day = my_market_hours.is_trading_day(date)
    
    print ("Date " + str(date) + " is trading day? -> ", is_trading_day)
        

Date 2019-07-23 is trading day? ->  True
Date 2019-07-24 is trading day? ->  True
Date 2019-07-25 is trading day? ->  True
Date 2019-07-26 is trading day? ->  True
Date 2019-07-27 is trading day? ->  False
Date 2019-07-28 is trading day? ->  False
Date 2019-07-29 is trading day? ->  True
Date 2019-07-30 is trading day? ->  True
Date 2019-07-31 is trading day? ->  True
Date 2019-08-01 is trading day? ->  True
Date 2019-08-02 is trading day? ->  True
Date 2019-08-03 is trading day? ->  False
Date 2019-08-04 is trading day? ->  False
Date 2019-08-05 is trading day? ->  True
Date 2019-08-06 is trading day? ->  True
Date 2019-08-07 is trading day? ->  True
Date 2019-08-08 is trading day? ->  True
Date 2019-08-09 is trading day? ->  True
Date 2019-08-10 is trading day? ->  False
Date 2019-08-11 is trading day? ->  False
Date 2019-08-12 is trading day? ->  True
Date 2019-08-13 is trading day? ->  True
Date 2019-08-14 is trading day? ->  True
Date 2019-08-15 is trading day? ->  True
Date 2019-